# Probabilities through monte carlo simulations

Here, rather than training a model to predict the $E[\text{dice roll score}]$, I just build a monte carlo simulation. 

In [1]:
import sys
sys.path.insert(0, '../../src')

In [2]:
import json
import random
from time import perf_counter
from collections import defaultdict

from farkle.logic import gameobjects as go

In [3]:
# for saving dicehands as json
class DiceHandEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, go.DiceHand):
            return obj.json_encode
        return super().default(obj)
    
def as_dicehand(dct):
    if '__DiceHand__' in dct:
        return go.DiceHand.json_decode(dct)
    return dct

In [7]:
class MonteCarloSimulation(object):
    def __init__(self, max_dice: int = 6):
        self.max_dice = max_dice
        self.dice_range = range(1, self.max_dice + 1)
        self.dice_hands = {i: go.DiceHand(num_dice=i) for i in self.dice_range}
        
        self.roll_obs = {i: 0 for i in self.dice_range}
        self.possible_score_obs = {i: 0 for i in self.dice_range}
        self.possible_score_frequencies = {i: defaultdict(int) for i in self.dice_range}
        
    def roll(self, num_dice: int):
        self.dice_hands[num_dice].roll()
        self.roll_obs[num_dice] += 1
        if self.dice_hands[num_dice].possible_scores():
            for dh in self.dice_hands[num_dice].possible_scores():
                dh: go.DiceHand
                self.possible_score_obs[num_dice] += 1
                self.possible_score_frequencies[num_dice][dh] += 1
        else:
            self.possible_score_frequencies[num_dice][None] += 1
            
    def roll_all(self, num_rolls: int = 1):
        """Rolls all hands in ratio where 1 dice is 1 roll and 
        each additional die is 6x more rolls"""
        for die_num in self.dice_range:
            num_sims = num_rolls * (6**(die_num-1))
            for roll_num in range(num_sims):
                self.roll(die_num)
                if roll_num == num_sims - 1:
                    sys.stdout.write(f'\rDie: {die_num} | ')
                    sys.stdout.write(f'{"#" * 20} | ')
                    sys.stdout.write(f'100%\n')
                elif int(roll_num * 100 / num_sims) % 5 == 0:
                    sys.stdout.write(f'\rDie: {die_num} | ')
                    sys.stdout.write(f'{"#" * int(roll_num * 20 / num_sims)}')
                    sys.stdout.write(f'{" " * (20 - int(roll_num * 20 / num_sims))} | ')
                    sys.stdout.write(f'{int(roll_num * 100 / num_sims)}%')
                
            sys.stdout.write('\n')
    
    def P_farkle(self, num_dice: int):
        return self.possible_score_frequencies[num_dice][None] / self.roll_obs[num_dice]
    
    def E_score(self, num_dice: int):
        E_score = 0
        for dh, freq in self.possible_score_frequencies[num_dice].items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs[num_dice]) * (1 - self.P_farkle(num_dice))
    
    # todo: probs best as an iter
    @property
    def _hash_frequencies(self):
        hash_to_dh = {}
        hash_to_freq = {}
        for d in self.dice_range:
            hash_to_freq[d] = {}
            for dh, freq in self.possible_score_frequencies[d].items():
                hash_to_dh[str(hash(dh))] = dh
                hash_to_freq[d][str(hash(dh))] = freq
        return hash_to_dh, hash_to_freq
    
    @staticmethod
    def _unhash_frequencies(hash_to_dh, hash_to_freq):
        _possible_score_freq = {}
        for d in hash_to_freq:
            d_int = int(d)
            _possible_score_freq[d_int] = defaultdict(int)
            for h, freq in hash_to_freq[d].items():
                if hash_to_dh[h] is not None:
                    dh = go.DiceHand.json_decode(hash_to_dh[h])
                else:
                    dh = None
                _possible_score_freq[d_int][dh] = int(freq)
        return _possible_score_freq
        
    @property
    def json_encode(self):
        hash_to_dh, hash_to_freq = self._hash_frequencies
        d = {'max_dice': self.max_dice, 
             'roll_obs': self.roll_obs, 
             'possible_score_obs': self.possible_score_obs, 
             'hash_to_dh': hash_to_dh, 
             'hash_to_freq': hash_to_freq}
             # 'possible_score_frequencies': self.possible_score_frequencies}
        return json.dumps(d, cls=DiceHandEncoder)
    
    def save(self, path: str):
        j = self.json_encode
        with open(path, 'w') as f:
            f.write(j)
        
    @staticmethod
    def load(path: str):
        with open(path, 'r') as f:
            j = json.loads(f.read()) #, object_hook=as_dicehand)
        
        mcs = MonteCarloSimulation(j['max_dice'])
        mcs.roll_obs = {int(k): v for k, v in j['roll_obs'].items()}
        mcs.possible_score_obs = {int(k): v for k, v in j['possible_score_obs'].items()}
        mcs.possible_score_frequencies = MonteCarloSimulation._unhash_frequencies(j['hash_to_dh'], 
                                                                                  j['hash_to_freq'])
        return mcs

In [8]:
# mcs = MonteCarloSimulation()
prob_path = '../../../models/possible_score_frequencies.json'
mcs = MonteCarloSimulation.load(prob_path)
print(mcs.roll_obs)

{1: 16, 2: 96, 3: 576, 4: 3456, 5: 20736, 6: 124416}


In [9]:
t0 = perf_counter()
mcs.roll_all(3)
t1 = perf_counter()
print(f'{round((t1-t0) / 60, 2)} minutes to run')

mcs.save(prob_path)

Die: 1 | #################### | 100%

Die: 2 | #################### | 100%

Die: 3 | #################### | 100%

Die: 4 | #################### | 100%

Die: 5 | #################### | 100%

Die: 6 | #################### | 100%

21.91 minutes to run


In [10]:
mcs.roll_obs

{1: 19, 2: 114, 3: 684, 4: 4104, 5: 24624, 6: 147744}

In [12]:
mcs.possible_score_frequencies[2]

defaultdict(int,
            {DiceHand(free=[1], locked=[], score=100): 40,
             DiceHand(free=[5], locked=[], score=50): 45,
             None: 43,
             DiceHand(free=[1, 1], locked=[], score=200): 5,
             DiceHand(free=[1, 5], locked=[], score=150): 14,
             DiceHand(free=[5, 5], locked=[], score=100): 3})

In [4]:
class MonteCarloDiceHand(go.DiceHand):
    def __init__(self, num_dice):
        super().__init__(num_dice=num_dice)

        self.possible_score_frequencies = defaultdict(int)
        self.roll_obs = 0
        self.possible_score_obs = 0

    def roll(self):
        super().roll()
        self.roll_obs += 1
        # save resulting possible scores if any otherwise None
        if self.possible_scores():
            for dh in self.possible_scores():
                dh: go.DiceHand
                self.possible_score_obs += 1
                self.possible_score_frequencies[dh] += 1
        else:
            self.possible_score_frequencies[None] += 1

    @property
    def E_any_score(self):
        return 1 - (self.possible_score_frequencies[None] / self.roll_obs)

    @property
    def E_score_arithmetic(self):
        E_score = 0
        for dh, freq in self.possible_score_frequencies.items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs) * self.E_any_score

In [5]:
mcdh = MonteCarloDiceHand(3)

In [6]:
mcdh

DiceHand(4, 4, 4, score=0)

In [7]:
mcdh.roll()

In [8]:
mcdh

DiceHand(1, 2, 2, score=0)

In [9]:
mcdh.possible_score_frequencies

defaultdict(int, {DiceHand(1, score=100): 1})

In [10]:
for i in range(4):
    mcdh.roll()

In [11]:
mcdh.possible_score_frequencies

defaultdict(int,
            {DiceHand(1, score=100): 1,
             None: 3,
             DiceHand(5, score=50): 1,
             DiceHand(5, 5, score=100): 1})

In [13]:
mcdh.roll_obs

5

In [14]:
mcdh.E_any_score

0.4

In [15]:
mcdh.E_score_arithmetic

33.333333333333336